<a href="https://colab.research.google.com/github/rohitpaul23/decisiontree_randomforest/blob/main/decisionTree.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DECISION TREE


Problem:

For this part of the assignment we will work on Bank Marketing dataset. We have been provided with a pre-defined set of test, train and validation of dataset to work with. In this dataset, the first row represents the labels of each column such that the last column represents whether the client subscribed (y/n) and following that, each row represents an example. 
We have to implement the decision tree algorithm for predicting whether client subscribed a term deposit

https://archive.ics.uci.edu/ml/datasets/Bank+Marketing

In [1]:
import numpy as np
import pandas as pd

In [2]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [4]:
bankTrainData = pd.read_csv('/content/gdrive/My Drive/projectData/ass3/q1/data/bank_dataset/bank_train.csv', header=0, delimiter=';')
bankTestData = pd.read_csv('/content/gdrive/My Drive/projectData/ass3/q1/data/bank_dataset/bank_test.csv', header=0, delimiter=';')
bankTrainData

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,57,unemployed,married,secondary,no,890,no,no,cellular,5,feb,343,4,-1,0,unknown,no
1,56,technician,married,secondary,no,2558,no,no,unknown,19,jun,288,1,-1,0,unknown,no
2,50,technician,married,tertiary,no,267,yes,no,cellular,21,nov,30,1,-1,0,unknown,no
3,47,management,married,unknown,no,4567,no,no,telephone,31,jul,921,4,-1,0,unknown,no
4,49,management,married,tertiary,no,5887,no,no,cellular,2,jun,181,3,293,2,failure,yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36163,37,blue-collar,married,secondary,no,128,yes,no,telephone,8,jul,379,3,-1,0,unknown,no
36164,43,technician,single,secondary,no,593,yes,no,unknown,5,may,55,1,-1,0,unknown,no
36165,32,technician,single,secondary,no,145,no,no,cellular,6,aug,135,2,-1,0,unknown,no
36166,35,technician,married,secondary,no,-215,no,yes,cellular,31,jul,57,34,-1,0,unknown,no


In [5]:
m = bankTrainData.shape[0]
n = bankTrainData.shape[1]
testRow = bankTestData.shape[0]
m, n, testRow

(36168, 17, 4521)

We got a total of 36168 training data, each having 17 columns(i.e., 16 features and 1 labels)

In [6]:
XTrain = bankTrainData.iloc[:,:-1]
yTrain = bankTrainData.iloc[:,-1]
XTest = bankTestData.iloc[:,:-1]
yTest = bankTestData.iloc[:,-1]
XTrain.shape

(36168, 16)

## Decision Tree Construction 


Construct a decision tree using the given data to predict which clients subscribed a term deposit. You should use mutual information as the criteria for selecting the attribute to split on. At each node, you should select the attribute which results in maximum decrease in the entropy of the class variable (i.e. has the highest mutual information with respect to the class variable). 

This problem has some attributes as numerical and some as categorical. For handling numerical attributes, you should use the following procedure. At any given internal node of the tree, a numerical attribute is considered for a two way split by calculating the median attribute value from the data instances coming to that node, and then computing the information gain if the data was split based on whether the numerical value of the attribute is greater than the median or not. 

For example, if we have have 10 instances coming to a node, with values of an attribute being
1(0,0,0,1,1,2,2,2,3,4) in the 10 instances, then we will split on value 1 of the attribute (median). At
any step, choose the attribute which results in highest mutual information by splitting on its median
value as described above. For categorical attributes, you can use the following two strategies 
(a) We can do a multi-way split by creating one decision tree branch for each possible value of the attribute.
(b) We can do a two way split by first converting the attribute to a one-hot encoding, and then,
treating each one-hot value as a separate Boolean valued attribute. 

In [7]:
featuresName = XTrain.keys()
featuresName

Index(['age', 'job', 'marital', 'education', 'default', 'balance', 'housing',
       'loan', 'contact', 'day', 'month', 'duration', 'campaign', 'pdays',
       'previous', 'poutcome'],
      dtype='object')

In [8]:
XTrain.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36168 entries, 0 to 36167
Data columns (total 16 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   age        36168 non-null  int64 
 1   job        36168 non-null  object
 2   marital    36168 non-null  object
 3   education  36168 non-null  object
 4   default    36168 non-null  object
 5   balance    36168 non-null  int64 
 6   housing    36168 non-null  object
 7   loan       36168 non-null  object
 8   contact    36168 non-null  object
 9   day        36168 non-null  int64 
 10  month      36168 non-null  object
 11  duration   36168 non-null  int64 
 12  campaign   36168 non-null  int64 
 13  pdays      36168 non-null  int64 
 14  previous   36168 non-null  int64 
 15  poutcome   36168 non-null  object
dtypes: int64(7), object(9)
memory usage: 4.4+ MB


Feature Engineering, convertingf object data type into integer

In [9]:
def oneHotCoding(feature):
    m = len(feature)
    discreteVal = feature.unique()
    len_ = len(discreteVal)
    multiArray = []
    for i in range(m):
        rowList = []
        for j in range(len_):
            if feature[i] == discreteVal[j]:
                rowList.append(1)
            else:
                rowList.append(0)
        multiArray.append(rowList)
    #print(multiArray)
    df = pd.DataFrame(multiArray, columns = discreteVal)
    return df

In [10]:
for feature in featuresName:
    if XTrain[feature].dtype != np.int64:
        df = oneHotCoding(XTrain[feature])
        commonName = set(XTrain.keys()).intersection(set(df.keys()))
        if commonName:    
            for name in commonName:
                i = 1
                while(True):
                    newName = name + str(i)
                    if newName in XTrain.keys():
                        i+=1
                    else:
                        df.rename(columns = {name:newName}, inplace=True)
                        break
        XTrain = pd.concat([XTrain, df], axis=1)
        XTrain.drop([feature], axis=1, inplace=True)
newFeaturesName = XTrain.keys()
newFeaturesName

Index(['age', 'balance', 'day', 'duration', 'campaign', 'pdays', 'previous',
       'unemployed', 'technician', 'management', 'blue-collar', 'admin.',
       'services', 'self-employed', 'retired', 'housemaid', 'student',
       'entrepreneur', 'unknown', 'married', 'divorced', 'single', 'secondary',
       'tertiary', 'unknown1', 'primary', 'no', 'yes', 'no1', 'yes1', 'no2',
       'yes2', 'cellular', 'unknown2', 'telephone', 'feb', 'jun', 'nov', 'jul',
       'may', 'aug', 'apr', 'dec', 'jan', 'oct', 'mar', 'sep', 'unknown3',
       'failure', 'other', 'success'],
      dtype='object')

Column with same names being modified by appendind literal behind it, features like 'unknown', 'yes' and 'no'

In [11]:
XTrain

,age,balance,day,duration,campaign,pdays,previous,unemployed,technician,management,...,apr,dec,jan,oct,mar,sep,unknown3,failure,other,success
0,57,890,5,343,4,-1,0,1,0,0,...,0,0,0,0,0,0,1,0,0,0
1,56,2558,19,288,1,-1,0,0,1,0,...,0,0,0,0,0,0,1,0,0,0
2,50,267,21,30,1,-1,0,0,1,0,...,0,0,0,0,0,0,1,0,0,0
3,47,4567,31,921,4,-1,0,0,0,1,...,0,0,0,0,0,0,1,0,0,0
4,49,5887,2,181,3,293,2,0,0,1,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36163,37,128,8,379,3,-1,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
36164,43,593,5,55,1,-1,0,0,1,0,...,0,0,0,0,0,0,1,0,0,0
36165,32,145,6,135,2,-1,0,0,1,0,...,0,0,0,0,0,0,1,0,0,0
36166,35,-215,31,57,34,-1,0,0,1,0,...,0,0,0,0,0,0,1,0,0,0


In [12]:
np.unique(XTrain['pdays'])

array([ -1,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  12,  13,
        14,  15,  17,  18,  19,  20,  21,  22,  24,  26,  27,  28,  29,
        30,  31,  33,  34,  35,  36,  37,  38,  39,  40,  41,  42,  43,
        44,  45,  46,  47,  48,  49,  50,  51,  52,  53,  54,  55,  56,
        57,  58,  59,  60,  61,  62,  63,  64,  65,  66,  67,  68,  69,
        70,  71,  72,  73,  74,  75,  76,  77,  78,  79,  80,  81,  82,
        83,  84,  85,  86,  87,  88,  89,  90,  91,  92,  93,  94,  95,
        96,  97,  98,  99, 100, 101, 102, 103, 104, 105, 106, 107, 108,
       109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121,
       122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134,
       135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147,
       148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160,
       161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173,
       174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 18

Converting y categorical value from ['no', 'yes'] to [0, 1]

In [13]:
yTrain.replace([ele for ele in yTrain.unique()], [ele for ele in range(len(yTrain.unique()))], inplace= True)

In [14]:
yTrain

0        0
1        0
2        0
3        0
4        1
        ..
36163    0
36164    0
36165    0
36166    0
36167    0
Name: y, Length: 36168, dtype: int64

## Using the Concept of Mutuao Information for decision Tree

In [16]:
def calculateEntropy(yLabel):
    totalLen = len(yLabel)
    label0 = yLabel[yLabel==0]
    lenLabel0 = len(label0)
    label1 = yLabel[yLabel==1]
    lenLabel1 = len(label1)
    
    probLabel0 = lenLabel0/totalLen
    probLabel1 = lenLabel1/totalLen
    if probLabel0 == 0 and probLabel1 == 0:
        return 0
    elif probLabel0 == 0:
        return -probLabel1*np.log(probLabel1)
    elif probLabel1 == 0:
        return -probLabel0*np.log(probLabel0)
    else:
        ent = -probLabel0*np.log(probLabel0) - probLabel1*np.log(probLabel1)
        return ent
    

In [17]:
calculateEntropy(yTrain)

0.35969285154524766

In [18]:
def mutualInfo(X, featureName, y):
    discreteVal = np.unique(X[featureName])
    k = len(discreteVal)
    
    if k == 2:
        indexX0 = list(X[X[featureName]==discreteVal[0]].index)
        indexX1 = list(X[X[featureName]==discreteVal[1]].index)
        ySeriesX0 = y[indexX0]
        ySeriesX1 = y[indexX1]
    else:
        med = np.median(X[featureName])
        indexX0 = list(X[X[featureName]>med].index)
        indexX1 = list(X[X[featureName]<=med].index)
        ySeriesX0 = y[indexX0]
        ySeriesX1 = y[indexX1]
    
    totalLen = len(y)
    label0 = y[y==0]
    lenLabel0 = len(label0)
    label1 = y[y==1]
    lenLabel1 = len(label1)

    probLabel0 = lenLabel0/totalLen
    probLabel1 = lenLabel1/totalLen

    if len(ySeriesX0) < 100 or len(ySeriesX0) < 100:
        return -1
    H_y = calculateEntropy(y)
    H_yX0 = calculateEntropy(ySeriesX0)
    H_yX1 = calculateEntropy(ySeriesX1)
    mi = H_y - (probLabel0*H_yX0 + probLabel1*H_yX1)
    return mi

In [19]:
def getFeature(featureList, X, y):
    max_ = 0
    featureName = ''
    for feature in featureList:
        mi = mutualInfo(X, feature, y)
        if mi > max_:
            max_ = mi
            featureName = feature
    return featureName

In [20]:
getFeature(newFeaturesName, XTrain, yTrain)

'cellular'

Defining a class node which will be used as a buliding blocks for constructing decision tree

In [21]:
class Node:
    def __init__(self, name=None, nodeType=None, left=None, right=None):
        self.name = name
        self.type = nodeType
        self.leftName = left
        self.leftnode = None
        self.rightName = right
        self.rightnode = None

class Tree:
    def __init__(self):
        self.head = None

In [22]:
atree = Tree()
atree.head = Node('A', 0, 1)
atree.head.leftnode = Node('B', 0, 1)
atree.head.rightnode = Node('C', 0, 1)

In [23]:
acopy = newFeaturesName.copy()
acopy = acopy.drop('age')
acopy

Index(['balance', 'day', 'duration', 'campaign', 'pdays', 'previous',
       'unemployed', 'technician', 'management', 'blue-collar', 'admin.',
       'services', 'self-employed', 'retired', 'housemaid', 'student',
       'entrepreneur', 'unknown', 'married', 'divorced', 'single', 'secondary',
       'tertiary', 'unknown1', 'primary', 'no', 'yes', 'no1', 'yes1', 'no2',
       'yes2', 'cellular', 'unknown2', 'telephone', 'feb', 'jun', 'nov', 'jul',
       'may', 'aug', 'apr', 'dec', 'jan', 'oct', 'mar', 'sep', 'unknown3',
       'failure', 'other', 'success'],
      dtype='object')

In [24]:
def DecisionTree(X, y, featureList):
    if X.shape[0] < 1000:
        numOfY = len(y)
        numOfY0 = len(y[y==0])
        numOfY1 = len(y[y==1])
        probY0 = numOfY0/numOfY
        probY1 = numOfY1/numOfY
        label = 0 if probY0>probY1 else 1
        anode = Node(label, 'Classification Node')
        return anode
    feature = getFeature(featureList, X, y)
    #print(feature)
    discreteVal = np.unique(X[feature])
    k = len(discreteVal)

    if k == 2:
        firstNodeX = X[X[feature]==discreteVal[0]].copy()
        firstNodey = y[X[feature]==discreteVal[0]].copy()
        secondNodeX = X[X[feature]==discreteVal[1]].copy()
        secondNodey = y[X[feature]==discreteVal[1]].copy()
        anode = Node(feature, 'Discrete Decision Node', discreteVal[0], discreteVal[1])
    else:
        med = np.median(X[feature])
        firstNodeX = X[X[feature]<med].copy()
        firstNodey = y[X[feature]<med].copy()
        secondNodeX = X[X[feature]>=med].copy()
        secondNodey = y[X[feature]>=med].copy()    
        anode = Node(feature, 'Continuous Decision Node', 'lt'+str(med), 'gteq'+str(med))

    featureList = featureList.drop(feature)
    
    firstNodeX.drop([feature], axis=1, inplace=True)
    secondNodeX.drop([feature], axis=1, inplace=True)
    anode.leftnode = DecisionTree(firstNodeX, firstNodey, featureList)
    anode.rightnode = DecisionTree(secondNodeX, secondNodey, featureList)

    return anode

In [25]:
decTree = DecisionTree(XTrain, yTrain, newFeaturesName)

In [26]:
for feature in featuresName:
    if XTest[feature].dtype != np.int64:
        df = oneHotCoding(XTest[feature])
        commonName = set(XTest.keys()).intersection(set(df.keys()))
        if commonName:    
            for name in commonName:
                i = 1
                while(True):
                    newName = name + str(i)
                    if newName in XTest.keys():
                        i+=1
                    else:
                        df.rename(columns = {name:newName}, inplace=True)
                        break
        XTest = pd.concat([XTest, df], axis=1)
        XTest.drop([feature], axis=1, inplace=True)
newTestFeaturesName = XTest.keys()
newTestFeaturesName

Index(['age', 'balance', 'day', 'duration', 'campaign', 'pdays', 'previous',
       'blue-collar', 'technician', 'self-employed', 'admin.', 'management',
       'student', 'services', 'entrepreneur', 'housemaid', 'unknown',
       'unemployed', 'retired', 'married', 'single', 'divorced', 'secondary',
       'primary', 'tertiary', 'unknown1', 'no', 'yes', 'yes1', 'no1', 'yes2',
       'no2', 'cellular', 'unknown2', 'telephone', 'apr', 'jun', 'jul', 'feb',
       'aug', 'may', 'jan', 'nov', 'sep', 'dec', 'oct', 'mar', 'unknown3',
       'other', 'success', 'failure'],
      dtype='object')

In [27]:
yTest.replace([ele for ele in yTest.unique()], [ele for ele in range(len(yTest.unique()))], inplace= True)

In [28]:
def prediction(checkX, temp):
    while(temp):
        if temp.type == 'Classification Node':
            return temp.name
        elif temp.type == 'Discrete Decision Node':
            if checkX[temp.name] == temp.leftName:
                temp = temp.leftnode
            else:
                temp = temp.rightnode
        else:
            sep = temp.leftName[2:]
            if checkX[temp.name] < float(sep):
                temp = temp.leftnode
            else:
                temp = temp.rightnode
    

Prediction for first test data with its actual label 

In [29]:
prediction(XTest.iloc[0], decTree), yTest.iloc[0]

(0, 0)

Computing Accuracy on Training Data

In [30]:
truePositive = 0
for i in range(m):
    yPred = prediction(XTrain.iloc[i], decTree)
    if yPred == yTrain.iloc[i]:
        truePositive += 1
accuracy = (truePositive/m)*100
accuracy

89.23910639239107

Computing Accuracy on Test Data

In [31]:
TestLength = XTest.shape[0]

In [33]:
truePositive = 0
for i in range(TestLength):
    yPred = prediction(XTest.iloc[i], decTree)
    if yPred == yTest.iloc[i]:
        truePositive += 1
accuracy = (truePositive/TestLength)*100
accuracy

88.25481088254811

Accuracy: 88.25%

--------------------------------------------------------------------------------------------------------------

## Decision Tree based on the concept of gain ratio (= gain(x)/split_info(x)) where gain(x)==mutual info(x)

In [34]:
len(XTrain)

36168

In [35]:
def split_info(XTrain, Xfeature):
    discreteVal = np.unique(XTrain[Xfeature])
    totalLen = len(XTrain)
    len_ = len(discreteVal)
    #print(len_)
    if len_ == 2:
        freqLeft = len(XTrain[XTrain[Xfeature]==discreteVal[0]][Xfeature])
        freqRight = len(XTrain[XTrain[Xfeature]==discreteVal[0]][Xfeature])
    elif len_>2:
        med = np.median(XTrain[Xfeature])
        #print(med)
        freqLeft = len(XTrain[XTrain[Xfeature]<=med][Xfeature])
        freqRight = len(XTrain[XTrain[Xfeature]>med][Xfeature])
        #print(freqLeft, freqRight)
    probLeft = freqLeft/totalLen
    probRight = freqRight/totalLen
    #print(Xfeature, probLeft, probRight)
    #print()
    if probLeft == 0 and probRight == 0:
        return 0
    elif probLeft == 0:
        return -probRight*np.log(probRight)
    elif probRight == 0:
        return -probLeft*np.log(probLeft)
    else:
        splitInfo = - probLeft*np.log(probLeft) - probRight*np.log(probRight)
        return splitInfo    

In [36]:
split_info(XTrain, 'pdays')

0.4735982624198676

In [37]:
def getFeatureUpdated(featureList, X, y):
    max_ = 0
    featureName = ''
    for feature in featureList:
        infoGain = mutualInfo(X, feature, y)
        splitInfo = split_info(X, feature)
        gainRatio = infoGain/splitInfo
        if gainRatio > max_:
            max_ = gainRatio
            featureName = feature
    return featureName

In [38]:
getFeatureUpdated(newFeaturesName, XTrain, yTrain)

'no'

In [39]:
def DecisionTreeC4_5(X, y, featureList):
    if X.shape[0] < 1000:
        numOfY = len(y)
        numOfY0 = len(y[y==0])
        numOfY1 = len(y[y==1])
        probY0 = numOfY0/numOfY
        probY1 = numOfY1/numOfY
        label = 0 if probY0>probY1 else 1
        anode = Node(label, 'Classification Node')
        return anode
    feature = getFeatureUpdated(featureList, X, y)
    #print(feature)
    discreteVal = np.unique(X[feature])
    k = len(discreteVal)

    if k == 2:
        firstNodeX = X[X[feature]==discreteVal[0]].copy()
        firstNodey = y[X[feature]==discreteVal[0]].copy()
        secondNodeX = X[X[feature]==discreteVal[1]].copy()
        secondNodey = y[X[feature]==discreteVal[1]].copy()
        anode = Node(feature, 'Discrete Decision Node', discreteVal[0], discreteVal[1])
    else:
        med = np.median(X[feature])
        firstNodeX = X[X[feature]<med].copy()
        firstNodey = y[X[feature]<med].copy()
        secondNodeX = X[X[feature]>=med].copy()
        secondNodey = y[X[feature]>=med].copy()    
        anode = Node(feature, 'Continuous Decision Node', 'lt'+str(med), 'gteq'+str(med))

    featureList = featureList.drop(feature)
    
    firstNodeX.drop([feature], axis=1, inplace=True)
    secondNodeX.drop([feature], axis=1, inplace=True)
    anode.leftnode = DecisionTree(firstNodeX, firstNodey, featureList)
    anode.rightnode = DecisionTree(secondNodeX, secondNodey, featureList)

    return anode

In [40]:
decTreeC4_5 = DecisionTreeC4_5(XTrain, yTrain, newFeaturesName)

In [41]:
truePositive = 0
for i in range(m):
    yPred = prediction(XTrain.iloc[i], decTreeC4_5)
    if yPred == yTrain.iloc[i]:
        truePositive += 1
accuracy = (truePositive/m)*100
accuracy

89.23910639239107

Accuracy: 89.239%

So theirs an improvement in the accuracy from 88.25% to 89.239%, using the concept of gain ratio

--------------------------------------------------------------------------------------------------------------


### Decision Tree Post Pruning 

As Training and Test accuracy is almost same, we can say the model is not overfit.

But if so then, One of the ways to reduce overfitting in decision trees is to grow the tree fully and then use post-pruning based on a validation set. In post-pruning, we greedily prune the nodes of the tree (and sub-tree below them) by iteratively picking a node to prune so that resultant tree gives maximum increase in accuracy on the validation set. In other words, among all the nodes in the tree, we prune the node such that pruning it (and sub-tree below it) results in maximum increase in accuracy over the validation set. This is repeated until any further pruning leads to decrease in accuracy over the validation set. 

## Random Forests:


Random Forests are extensions of decision trees, where we grow multiple decision trees in parallel on bootstrapped samples constructed from the original training data. A number of libraries are available for learning Random Forests over a given training data. In this particular question I used the scikit-learn library of Python to grow a Random Forest.

Experimented with the following parameter values (in the given range): 

(a) n estimators (50 to 450 in range of 100). 

(b) max f eatures (0.1 to 1.0 in range of 0.2) 

(c) min samples split (2 to 10 in range of


Used the out-of-bag accuracy to tune to the optimal values for these parameters. We should perform
a grid search over the space of parameters. Report training, out-of-bag, validation and test set accuracies for the optimal set of parameters obtained. How do your numbers, i.e., train, validation and test set accuracies compare
with those you obtained above?

One must note that if you encode categorical attributes to integers, scikit will assume they are ordinal.
For example, if (Male, Female, Others) is mapped to (0, 1, 2) scikit will treat them as integers with 1
<2 <3. So we may need to encode in a way this ordering can be avoided. Scikit uses two split for
handling categorical

In [42]:
from sklearn.ensemble import RandomForestClassifier

In [43]:
clf25 = RandomForestClassifier(n_estimators=25, criterion="entropy", min_samples_split=50)
clf25.fit(XTrain, yTrain)

RandomForestClassifier(criterion='entropy', min_samples_split=50,
                       n_estimators=25)

In [44]:
XTest = XTest[newFeaturesName]

In [45]:
clf25YPred = clf25.predict(XTest)
clf25YPred

array([0, 0, 0, ..., 0, 0, 0])

In [46]:
def calcAccuracy(clfpred, realy, len_):
    truePositive = 0
    for i in range(len_):
        if clfpred[i] == realy.iloc[i]:
            truePositive += 1
    accuracy = (truePositive/len_)*100
    return accuracy

In [47]:
calcAccuracy(clf25YPred, yTest, TestLength)

89.8252598982526

Using Gini as a criterion

In [48]:
clf25gini = RandomForestClassifier(n_estimators=25, criterion="gini", min_samples_split=50)
clf25gini.fit(XTrain, yTrain)

RandomForestClassifier(min_samples_split=50, n_estimators=25)

In [49]:
clf25giniYPred = clf25gini.predict(XTest)
clf25giniYPred

array([0, 0, 0, ..., 1, 0, 0])

In [50]:
calcAccuracy(clf25giniYPred, yTest, TestLength)

90.02433090024331

Comparitively giving us the similar result

So by using 'Random Forest' the accuracy improved from 88.25% to 90%